In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# to enable the google-cloud sdk "module", install the google-cloud-bigquery client library in shell/command prompt
# pip install --upgrade google-cloud-bigquery
# check google.cloud.bigquery version with: 
# pip show google-cloud
from google.cloud import bigquery

In [3]:
# check in which path your current working directory is
# os.getcwd()import datetime as datetime

In [4]:
import datetime as datetime
client = bigquery.Client.from_service_account_json('<placeholder>') #populate with your bigQuery service account

now = datetime.datetime.now()
PROJECT=""
DATASET=""
QUERY=""
START_DATE = (now - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
END_DATE = (now - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
base_path = ""
filename="london_crime.sql"

In [5]:
# target table id
t_id = 'analytics_summit_'+END_DATE.replace("-","")
print(t_id)

analytics_summit_20181108


In [6]:
bq_date=now.strftime("%Y-%m-%d")
print(START_DATE)
print(END_DATE)

2018-11-02
2018-11-08


In [7]:
#Add function for BQ client
def bigquery_job ():
    #query config
    path_to_file = os.path.join(base_path, filename)
    fd = open(path_to_file , 'r')
    q=fd.read()
    query = q.format(start_date=START_DATE,end_date=END_DATE,dataset=DATASET,query=QUERY)
    #print(query)
    #connection config
    query_job = bigquery.QueryJobConfig()
    query_job.use_legacy_sql = False
    
    start=datetime.datetime.now()
    print('start: ' + str(start))
    query = client.query(query, job_config=query_job)
    result = query.result(timeout=None)
    dataframe = result.to_dataframe()
    bytesProcessed = query.total_bytes_processed
    jobId = query.job_id
    end=datetime.datetime.now()
    print('end: ' + str(end))
    print('duration: ' + str((end-start)))
    print('processed bytes: '+ str(bytesProcessed) 
          + ', hexa GB: ' + str(float(bytesProcessed/10**9))
          + ', in GiB: ' + str(float(bytesProcessed/2**30)) 
          + ', in TB: ' + str(float(bytesProcessed/2**40)))
    print('job ID: '+jobId)
    return dataframe

In [8]:
sample = bigquery_job()

start: 2018-11-09 22:39:23.460315
end: 2018-11-09 22:39:29.954995
duration: 0:00:06.494680
processed bytes: 629125272, hexa GB: 0.629125272, in GiB: 0.5859185680747032, in TB: 0.0005721861016354524
job ID: a9d0045e-b19c-440d-ba73-982b4680ab2e


In [9]:
# print a preview of the dataframe's first rows
sample.head()

,borough,major_category,total_value
0,Westminster,Theft and Handling,27520
1,Camden,Theft and Handling,14088
2,Lambeth,Theft and Handling,13155
3,Southwark,Theft and Handling,12946
4,Islington,Theft and Handling,12077


In [10]:
# IF running on local machine or not having installed any further packages, 
# I installed 'pyarrow' to have the insert job running, as proposed by the resulting error if not done so.
# Alternatively it proposed 'fastparquet'
# here is the code to drop in the terminal for the pyarrow package (installation worked with Python 3.7 Windows 64-bit):
#     conda install -c conda-forge pyarrow 

In [13]:
# function to write query to a table in above defined dataset
def bigquery_insert ():
    project_id ='<placeholder>'
    dataset_id = 'analytics_summit'
    table_id = t_id
    #client = bigquery.Client.from_service_account_json('')   #populate with your bigQuery service account

    #query config
    path_to_file = os.path.join(base_path, filename)
    fd = open(path_to_file , 'r')
    q=fd.read()
    query = q.format(start_date=START_DATE,end_date=END_DATE,dataset=DATASET,query=QUERY)

    job_config = bigquery.QueryJobConfig()
    # Set the destination table
    client.project = project_id
    table_ref = client.dataset(dataset_id).table(table_id)
    job_config.destination = table_ref

    
    trial_job = client.load_table_from_dataframe(sample, table_ref)

    trial_job.result()  # Waits for the query to finish
    print('Query results loaded to table {}'.format(table_ref.path))

In [14]:
bigquery_insert()

Query results loaded to table /projects/tough-zoo-22/datasets/analytics_summit/tables/analytics_summit_20181108
